# **Certificate Manager Agent**

## **About the Scenario**
This scenario demonstrates how to build an AI-powered Certification Manager Agent using Azure AI Agent Service. The agent is designed to analyze certification data stored in Excel files, convert them to a compatible format, and enable intelligent querying and comparison of certification offerings from various providers. By integrating file search, function calling, and Bing search capabilities, the agent can answer user questions, compare internal data with external sources, and highlight any discrepancies or insights. This workflow showcases the end-to-end process of preparing data, configuring the agent, running queries, and extracting actionable insights—all within a secure and reproducible environment.

## **Data**
This scenario uses a certification dataset stored in the [`data/`](./data/) folder of this repository. The primary file, `workday_data.xlsx`, contains detailed information about certification providers, certification names, and related guidelines.

Make sure you have the following file in the `data/` directory before running the notebook:

- workday_data.xlsx

This file will be processed and converted to a Markdown table for compatibility with Azure AI Agent Service. The agent will use this data to analyze, compare, and answer queries about certification offerings from various providers.


## **Before you begin**

#### Step 1: Install required libraries
Installing dependencies directly within a Jupyter notebook is a good practice because it ensures that all required packages are installed in the correct versions, making the notebook self-contained and reproducible. This approach helps other users or collaborators to set up the environment quickly and avoid potential issues related to missing or incompatible packages.

In [230]:
# Install the packages
%pip install -r ./requirements.txt --upgrade
print("\nPackages installed successfully.")

Note: you may need to restart the kernel to use updated packages.

Packages installed successfully.


#### Step 2: Setting up the environment
Before we begin, we need to load the necessary environment variables from a `.env` file. These variables include sensitive information such as API keys and endpoint URLs, which are crucial for running the code successfully.

Here’s what you need to do:
- Ensure your `.env` file is properly configured in the `.venv/.env` format. We have provided an template `.env` file, `.env.example` for your reference.
- Verify that all required secrets are included in the file before running the code.


The `.env` file must contain the following secrets:
- PROJECT_CONNECTION_STRING: URL to connect to the Azure AI Agent Service Project to access project resources.
- AZURE_OPENAI_DEPLOYMENT: The name of the Azure AI Agent Service model deployment.

Now, let’s load these variables and get started!

<code style="background:yellow;color:black">Note: Make sure to keep your `.env` file secure and avoid sharing it publicly. </code>

*For more information about leveraging Python Virtual Environments can be found [here](https://docs.python.org/3/library/venv.html).*

In [252]:
import json

import pandas as pd

def get_information_about_certification_provider(certification_provider: str) -> json:
    """
    Fetches information about a certifications available from the certification provider from an Excel file.

    This function reads an Excel file containing certification data, groups the data by 
    the "OLD Certifier or Issuer" column, and maps each certifier to a list of 
    corresponding "Duplicate Certification Name" values. It then retrieves the 
    certifications for the specified certification provider.

    If the specified certification provider is not found in the data, the function 
    returns a JSON object with an error message.

    :param certification_provider: The name of the certification provider to look up.
    :type certification_provider: str

    :return: A JSON string containing the certifications for the specified provider, 
             or an error message if the provider is not found.
    :rtype: str

    Example:
    --------
    Suppose the Excel file contains the following data:

    | OLD Certifier or Issuer | Duplicate Certification Name |
    |--------------------------|-----------------------------|
    | Certifier A             | Certification 1            |
    | Certifier A             | Certification 2            |
    | Certifier B             | Certification 3            |

    Calling the function with "Certifier A" will return:
    {
        "certifications": ["Certification 1", "Certification 2"]
    }

    Calling the function with "Certifier C" will return:
    {
        "error": "Certification provider not found."
    }
    """

    file_path = "data/workday_data.xlsx"
    df = pd.read_excel(file_path)
    # Group by "OLD Certifier or Issuer" and aggregate "Duplicate Certification Name"
    mapping = (
        df.groupby("OLD Certifier or Issuer")["Duplicate Certification Name"]
        .apply(list)
        .to_dict()
    )
    # print(f"Mapping for {mapping}")

    # print(mapping[certification_provider])
    # If the certification provider is not found, return an empty JSON object
    if certification_provider not in mapping:
        return json.dumps({"error": "Certification provider not found."})
    else:
        # Return the certifications as a JSON string
        return json.dumps({"certifications": mapping[certification_provider]})


# def fetch_weather(location: str) -> str:
#     """
#     Fetches the weather information for the specified location.

#     :param location: The location to fetch weather for.
#     :return: Weather information as a JSON string.
#     """
#     # Mock weather data for demonstration purposes
#     mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
#     weather = mock_weather_data.get(location, "Weather data not available for this location.")
#     return json.dumps({"weather": weather})


#print(get_information_about_certification_provider("365 Data Science"))
# Define user functions
user_functions = {get_information_about_certification_provider}

In [253]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the secrets
__PROJECT_CONNECTION_STRING = os.getenv("PROJECT_CONNECTION_STRING")
__AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
print(__PROJECT_CONNECTION_STRING)
# Verify environment variables'AIProjectClient
if not all([__PROJECT_CONNECTION_STRING, __AZURE_OPENAI_DEPLOYMENT]):
    raise EnvironmentError("One or more environment variables are missing. Please check the .env file.")
else:
    print("Environment variables loaded successfully.")

https://azure-ai-foundry-pg.services.ai.azure.com/api/projects/foundry_demo
Environment variables loaded successfully.


## **Azure AI Agent Service Agent Setup**

### Step 1: Initializing the Azure AI Agent Service Project Client
First, we will initialize the Azure AI Agent Service Project client using Azure’s `DefaultAzureCredential` for authentication, allowing seamless integration with Azure resources. You will need to log into Azure using the Azure CLI.

In [254]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

try:
    # Initialize the Azure AI Project client
    project_client = AIProjectClient(
        credential=DefaultAzureCredential(),
        endpoint=__PROJECT_CONNECTION_STRING,
    )

    print("Azure AI Agent Service client created successfully.")

except Exception as e:
    print(f"Error creating the project client: {e}")

Azure AI Agent Service client created successfully.


### Step 2: Initializing the Azure Agent Client
Next, we’ll initialize the Azure Agent Runtime client. The Azure Agent Client serves as the interface to interact with Azure AI Agent Service services. 

In [255]:
agent_client = project_client.agents

print("Agent Client created successfully.")

Agent Client created successfully.


## **Data Processing**

### Step 1: Convert Excel files to Markdown
Azure AI Agent Service Agents require data in specific file formats for processing. Since Excel files aren’t natively supported for File Search, our files need to be converted to Markdown tables in separate Markdown files. This step involves:

- Reading Excel files using pandas.
- Writing the data into Markdown format for compatibility.

In [256]:
import pandas as pd
from pathlib import Path
from azure.ai.agents.models import (
    FilePurpose,
)
# Define paths
output_dir = "uploads"
data_dir_path = "data"
certification_data_files = ["workday_data.xlsx"]
output_dir_path = Path(data_dir_path) / output_dir

# Ensure output directory exists
if not Path(output_dir_path).exists():
    Path(output_dir_path).mkdir(parents=True, exist_ok=True)

# Prefix the data directory path to each file
certification_data_files = [Path(data_dir_path) / curr_file for curr_file in certification_data_files]

# Store the uploaded file IDs to be used later when enabling File Search
markdown_file_paths = []

for curr_file in certification_data_files:
    try:
        # Check if the file exists
        if not Path(curr_file).exists():
            raise FileNotFoundError(f"The file '{curr_file}' does not exist.")

        # Load the Excel File into a DataFrame
        df = pd.read_excel(curr_file)
        print(f"Workbook '{curr_file}' successfully loaded.")

        # Get the base name of the Excel file without the extension
        base_name = Path(curr_file).stem

        # Convert the DataFrame to a Markdown table
        md_tbl_str = df.to_markdown(index=False, tablefmt="pipe")

        # Define the output file name
        output_file = Path(output_dir_path) / f"{base_name}.md"
        markdown_file_paths.append(output_file)

        # Write the Markdown table to the file
        with Path(output_file).open("w") as f:
            f.write(md_tbl_str)

        print(f"Markdown file '{output_file}' successfully written.")

    except FileNotFoundError as e:
        print(f"Error: {e}")

    except Exception as e:
        print(f"An error occurred while processing the Excel file '{curr_file}': {e}")

Workbook 'data/workday_data.xlsx' successfully loaded.
Markdown file 'data/uploads/workday_data.md' successfully written.


### Step 2: Upload Markdown file(s) to OpenAI
Next, we'll upload the markdown files from the local directory, `data\uploads`, to the Azure AI Agent Service Project Deployment. Before uploading, any markdown files in the project with the same name are removed to ensure the latest versions are used and to prevent duplicates. This step efficiently manages cleanup and file upload.

In [257]:
uploaded_file_ids = []

try:
    for local_file in os.listdir(output_dir_path):
        # Define the local file path
        local_file_path = Path(output_dir_path) / local_file

        # Check if the file is a Markdown file
        if not local_file.endswith(".md"):
            continue
        else:
            print(f"Processing File '{local_file_path}'...")

            # Check if the file already exists in the cloud, and delete it if it does
            # for cloud_file in agent_client.list_files().data:
            #     if cloud_file.filename == local_file:
            #         agent_client.delete_file(cloud_file.id)
            #         print(f"Deleted existing cloud file: '{cloud_file.filename}'")

            # Use the upload and poll SDK helper to upload the local file, add them to the vector store,
            # and poll the status of the file batch for completion.
            file = project_client.agents.files.upload_and_poll(
                file_path=local_file_path,
                purpose=FilePurpose.AGENTS
            )
            uploaded_file_ids.append(file.id)

            # TODO: Check if there is a status to look
            print(f"Uploaded file, file ID: {file.id}")

    print(f"File IDs: {uploaded_file_ids}")

except FileNotFoundError:
    print(f"Error: The file '{curr_file}' was not found.")
    raise

except Exception as e:
    print(f"An error occurred while processing the Excel file: {e}")
    raise

Processing File 'data/uploads/workday_data.md'...
Uploaded file, file ID: assistant-KZ1rauCdWbYr5GCeH84vbu
File IDs: ['assistant-KZ1rauCdWbYr5GCeH84vbu']


### Step 3: Create Vector Store
The Vector Store is used to store embeddings of uploaded files, enabling the File Search tool to efficiently locate relevant content.

In [258]:
# Create a vector store called "Financial Statements"

vector_store = agent_client.vector_stores.create_and_poll(
    file_ids=uploaded_file_ids,
    name="Certification Data Vector Store",)
print(f"Vector store '{vector_store.name} ({vector_store.id})' created successfully.")

Vector store 'Certification Data Vector Store (vs_XMAhOlBssxoeMTwyhiLz6q34)' created successfully.


### Step 4: Create File Search Tool
The File Search tool enables the AI agent to query the uploaded files. It uses the embeddings stored in the Vector Store to locate and retrieve relevant content.

In [259]:
from azure.ai.agents.models import FileSearchTool

# Create file search tool with resources followed by creating agent
file_search = FileSearchTool(vector_store_ids=[vector_store.id])
print(f"File search tool created successfully for Vector Store {vector_store.name} ({vector_store.id}).")

File search tool created successfully for Vector Store Certification Data Vector Store (vs_XMAhOlBssxoeMTwyhiLz6q34).


### Step 5: Bing Grounding
The File Search tool enables the AI agent to query the uploaded files. It uses the embeddings stored in the Vector Store to locate and retrieve relevant content.

In [260]:
project_client.connections.get(name="binggrounding").id

'/subscriptions/ae6cbacb-2eac-42cc-978e-516b8ef7628d/resourceGroups/rg-pragati-gupta-1/providers/Microsoft.CognitiveServices/accounts/azure-ai-foundry-pg/projects/foundry_demo/connections/binggrounding'

In [261]:
from azure.ai.agents.models import BingGroundingTool
bing_conn_name = os.getenv("BING_CONNECTION_NAME")
bing_connection = project_client.connections.get(name=bing_conn_name)
# Get the connection ID from the Bing connection
if not bing_connection:
    raise ValueError(f"Bing connection '{bing_conn_name}' not found. Please check the connection name.")
conn_id = bing_connection.id
print(f"Bing Connection ID: {conn_id}")
# Initialize the Bing Grounding tool
bing_tool = BingGroundingTool(connection_id=conn_id)


Bing Connection ID: /subscriptions/ae6cbacb-2eac-42cc-978e-516b8ef7628d/resourceGroups/rg-pragati-gupta-1/providers/Microsoft.CognitiveServices/accounts/azure-ai-foundry-pg/projects/foundry_demo/connections/binggrounding


## **Running the Azure AI Agent Service Agent**

### Step 1: Create the Agent

In [ ]:
from azure.ai.agents.models import FunctionTool
try:
    
    functions = FunctionTool(user_functions)
    project_client.agents.enable_auto_function_calls(functions)

    agent = project_client.agents.create_agent(
        model=__AZURE_OPENAI_DEPLOYMENT,
        name="Certification Manager Agent",
        instructions=f"""
You are a smart assistant with access to an uploaded Excel file containing certification data. The file has the following columns:

- NEW Certifier or Issuer  
- OLD Certifier or Issuer  
- NEW Certification Name  
- Duplicate Certification Name  
- GUIDELINES

You also have access to a function:  
`get_information_about_certification_provider(provider_name)`  
It returns a JSON-formatted string in the form:  
    {{"certifications": ["<certification_name_1>", "<certification_name_2>", ...]}}

### Your responsibilities:


1. **Handle User Queries**:
   - For general questions about the dataset (e.g., list of certifiers, all certifications, guidelines), use the Excel file directly.
   - If the user asks about a specific provider `<provider_name>`:
     a. Search NEW Certifier or Issuer columns for a match.
     b. If no match is found, respond:
        `"The provider <provider_name> was not found in the data."`  
        Then stop further actions.
     c. If found, call `get_information_about_certification_provider(<provider_name>)` to get certifications associated with that provider.

2. **External Data Comparison**:
   - Make a Bing search with the query:  
     `"List of all certifications provided by <provider_name>"`
   - Expect a plain **list of certification names** as the result.
  

3. **Compare and Report**:
   - Display the following:
     - Certifications from the internal function.
     - Certifications from the Bing call.


Always keep responses accurate, concise, and transparent. Never assume or fabricate information not present in either source.

# Example:
  # If the user asks: "What certifications does Microsoft offer?"
  # 1. Search for "Microsoft" in the NEW Certifier or Issuer column.
  # 2. If found, call: get_information_about_certification_provider("Microsoft")
  # 3. Make a Bing search: "List of all certifications provided by Microsoft"
  # 4. Compare the internal and Bing results, then display both lists.
""",

        tools=file_search.definitions + functions.definitions + bing_tool.definitions,
        tool_resources=file_search.resources,
    )
    print(f"Agent created successfully.({agent.id})")
except Exception as e:
    print("Error creating Agent:", e)

Agent created successfully.(asst_vW3tZb9cctcGrMbcXBjjMY2f)


### Step 2: Start a New Converstaion
Conversation threads in Azure AI Agent Service enable context-aware interactions, storing both user prompts and agent responses. This step creates a new thread for the Sales Analyst Agent to handle queries.

In [294]:
# Create a conversation thread
try:
    thread = agent_client.threads.create()
    print(f"Thread created successfully ({thread.id})")
except Exception as e:
    print("Error creating thread:", e)

Thread created successfully (thread_RIN9LKv5figMy4bLq2gdfQcn)


### Step 3: Query the AI Agent
Next, we’ll add a user message to the thread. The AI agent responds to a user-defined prompt, such as calculating revenue by region. It processes the prompt and retrieves the necessary data to deliver insights.

In [295]:
# Define the user question
prompt_content = "Which certification provided by Amazon"

# Add the question to the thread
try:
    message = agent_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt_content,
    )
    print(f"Successfully added User prompt to the thread. (Message ID {message.id})")
except Exception as e:
    print("Error adding user question:", e)

Successfully added User prompt to the thread. (Message ID msg_ndValgjW1CvGVv2FEBawweej)


### Step 4: Run the AI Agent
In this step, we instruct the AI agent to process the user’s query within the created thread. The agent analyzes the context, executes the required tools (e.g., File Search), and generates a response. The `create_and_proces_run` function triggers the agent to process the user's prompt and produce an agent output.

In [296]:
# Initiate the Agent's response
try:
    run = agent_client.runs.create_and_process(
        thread_id=thread.id,
        agent_id=agent.id,
    )
    print("Run started:", run.id)
except Exception as e:
    print("Error starting run:", e)

Run started: run_ZKizdT50oejGF8V6UrifCtCm


If the agent run fails, we will identify the issue. A common cause is exceeding the rate limit, requiring additional Azure quotas.

In [297]:
if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

### Step 5: Extract Insights
After the agent processes the prompt, we retrieve its responses, which contain the requested insights. This step ensures that the conversation thread is queried to fetch all relevant messages generated during the interaction.

In [298]:
# Retrieves all messages from the thread in ascending order after the user message
messages = agent_client.messages.list(thread_id=thread.id, order="asc", after=message.id)

print("Run completed!\n\nMESSAGES\n")


messages = agent_client.messages.list(thread_id=thread.id)
for msg in messages:
    for content in msg.content:
        if content["type"] == "text":
            print(content["text"]["value"])

Run completed!

MESSAGES

Amazon provides the following certifications:

1. **AWS Certified Solutions Architect – Professional**
2. **AWS Certified Solutions Architect Associate**
3. **AWS Solutions Training for Partners: Machine Learning on AWS - Business (Digital)**
4. **AWS Certified SysOps Administrator - Associate**
5. **AWS Certified Developer – Associate**
6. **AWS Certified DevOps Engineer - Professional**
7. **Certified Cloud Practitioner**
8. **Certified Solutions Architect, Associate**
9. **AWS Data Analytics - Specialty**
10. **AWS Certified Machine Learning - Specialty**
11. **AWS Certified Advanced Networking - Specialty**
12. **AWS Technical Professional**

These are directly sourced from your uploaded dataset【4:0†source】【4:1†source】【4:3†source】【4:5†source】【4:9†source】【4:11†source】. Let me know if you would like further details on any specific certification!
Which certification provided by Amazon


## **Clean Up**

Properly cleaning up Azure resources after completing the analysis is essential to maintain a tidy Azure AI Agent Service Project environment and to avoid incurring unnecessary costs.

### Step 1: Deleting the Vector Store
Remove the Vector Store to free up storage resources.

In [ ]:
# project_client.agents.vector_stores._delete_vector_store(vector_store.id)
# print("Deleted vector store")

Deleted vector store


### Step 2: Deleting Uploaded Files
Ensure all uploaded files are removed from Azure to maintain a clean environment.


In [ ]:
# # For each file id in the list, delete the file
# for file_id in uploaded_file_ids:
#     project_client.agents.files.delete(file_id)
#     print(f"Deleted file: {file_id}")

Deleted file: assistant-6PH86ohbBSAqEeJA4K8obg


### Step 3: Deleting the AI Agent
Remove the AI Agent to release associated resources.

In [249]:
agent.id

'asst_DBwk4CVMc9UWS7TtvbayRXOl'

In [ ]:
# # Get agent list
# agents = agent_client.list_agents()
# # Check if the agent exists in the list
# agent_exists = any(a.id == agent.id for a in agents)
# agent_name = "Certification Manager Agent"
# if agent_exists:
#     # If the agent exists in the list, delete it
#     print("Agent exists to delete.")
#     agent_client.delete_agent(agent_id=agent.id)

### Step 4: Deleting local Markdown files
Remove the local Markdown files generated during this scenario to maintain a clean local environment.

In [ ]:
# # Delete all local markdown files from the output directory
# for file_path in markdown_file_paths:
#     Path(file_path).unlink()
#     print(f"Deleted local markdown file: {file_path}")

Deleted local markdown file: data/uploads/workday_data.md
